In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_5002/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_11_9_0,0.998952,0.997703,0.998706,0.998144,0.126353,0.282348,0.072987,0.183825,"Hidden Size=[30], regularizer=0.01, learning_r..."
1,model_11_9_1,0.998887,0.996772,0.998349,0.997436,0.134236,0.396901,0.093136,0.253953,"Hidden Size=[30], regularizer=0.01, learning_r..."
2,model_11_9_2,0.998792,0.995795,0.997980,0.996696,0.145644,0.516922,0.113978,0.327302,"Hidden Size=[30], regularizer=0.01, learning_r..."
3,model_11_9_3,0.998678,0.994806,0.997609,0.995946,0.159397,0.638610,0.134912,0.401576,"Hidden Size=[30], regularizer=0.01, learning_r..."
24,model_11_9_4,0.998552,0.993825,0.997244,0.995204,0.174590,0.759165,0.155505,0.475090,"Hidden Size=[30], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
188,model_13_7_1,0.994974,0.998961,0.998979,0.998988,0.606103,0.236908,0.074458,0.160461,"Hidden Size=[35], regularizer=0.01, learning_r..."
197,model_11_5_6,0.994706,0.992524,0.996898,0.994703,0.638462,1.010897,0.406611,0.726530,"Hidden Size=[30], regularizer=0.01, learning_r..."
204,model_11_5_5,0.993970,0.992810,0.997061,0.994926,0.727213,0.972127,0.385197,0.695922,"Hidden Size=[30], regularizer=0.01, learning_r..."
206,model_13_7_0,0.993900,0.998803,0.999651,0.999013,0.735717,0.273016,0.025418,0.156499,"Hidden Size=[35], regularizer=0.01, learning_r..."
